<a href="https://colab.research.google.com/github/zephyris/tryptag/blob/main/examples/tryptools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install tryptag module

#@markdown Install the `tryptag` module using `pip`.

!pip install git+https://github.com/zephyris/tryptag

In [ ]:
#@title Setup tryptag

#@markdown The `tryptag` module handles all data retrieval. `CellLine` is used to pass cell line information to methods. `tryptools` provides iamge analysis tools.

# import and set up TryTag instance
from tryptag import TrypTag, CellLine, tryptools
tryptag = TrypTag()

In [ ]:
#@markdown Cells are opened using their `gene_id` (from [TriTrypDB](https://tritrypdb.org)) and tagged `terminus`, which must be `"n"` or `"c"`.
gene_id = "wild-type.1.0h" #@param {type:"string"}
terminus = "n" #@param ["n", "c"]
field_index = 2 #@param {type:"integer"}
cell_index = 8 #@param {type:"integer"}

# open the cell
cell_image = tryptag.open_cell(CellLine(gene_id, terminus), field_index, cell_index, width = -32)

#@markdown The opened cell image can be passed to various `tryptools` analysis methods.
#@markdown All coordinates in the measurements are the position in this cell image, and will depend on the `width` used when opened.

In [ ]:
#@title Signal analysis

#@markdown Simple analysis of median background-subtracted mNG signal.

result_signal = tryptools.cell_signal_analysis(cell_image)

print("- signal analysis result -")
for key in result_signal:
  print(key+":", result_signal[key])

In [ ]:
#@title KN analysis

#@markdown Simple analysis of kinetoplast and nucleus number and position.

result_kn = tryptools.cell_kn_analysis(cell_image)

print("- kn analysis result -")
for key in result_kn:
  if key == "objects_k" or key == "objects_n":
    print(key+":")
    for index, value in enumerate(result_kn[key]):
      for key2 in value:
        if key2 == "centroid":
          print("", key2+":", value[key2]["x"], ",", value[key2]["y"])
        else:
          print("", key2+":", value[key2])
  else:
    print(key+":", result_kn[key])

tryptools.plot_morphology_analysis(cell_image, result_kn)

In [ ]:
#@title Midline analysis

#@markdown Simple analysis of cell shape from a skeletonisation, giving a midline if skeletonised to a single line.

result_midline = tryptools.cell_midline_analysis(cell_image)

print("- midline analysis result -")
for key in result_midline:
  if key == "midline":
    print(key+":")
    print("", "[", "coordinates list of length", len(result_midline), "]")
  else:
    print(key+":", result_midline[key])

tryptools.plot_morphology_analysis(cell_image, result_midline)

In [ ]:
#@title Morphology analysis

#@markdown Combined midline and KN analysis: If cell skeletonised to a single line then gives an overall cell morphology along the anterior posterior axis.

result_morphology = tryptools.cell_morphology_analysis(cell_image)

print("- morphology analysis result -")
for key in result_morphology:
  if key == "objects_k" or key == "objects_n":
    print(key+":")
    for index, value in enumerate(result_morphology[key]):
      for key2 in value:
        if key2 == "centroid":
          print("", key2+":", value[key2]["x"], ",", value[key2]["y"])
        else:
          print("", key2+":", value[key2])
  elif key == "midline":
    print(key+":")
    print("", "[", "coordinates list of length", len(result_midline), "]")
  elif key == "distance":
    print(key+":")
    print("", "[", "midline distance list of length", len(result_midline), "]")
  else:
    print(key+":", result_morphology[key])

tryptools.plot_morphology_analysis(cell_image, result_morphology)

if "midline" in result_morphology:
  print("")
  print("- interpreted summary -")
  print("cell length:", result_morphology["distance"][-1], "px")
  if "objects_k" in result_morphology:
    print("posterior - kinetoplast distance:")
    for k in range(len(result_morphology["objects_k"])):
      print("", "k", k+1, result_morphology["distance"][result_morphology["objects_k"][k]["midline_index"]], "px")
  if "objects_n" in result_morphology:
    print("posterior - nucleus distance:")
    for n in range(len(result_morphology["objects_n"])):
      print("", "n", n+1, result_morphology["distance"][result_morphology["objects_n"][n]["midline_index"]], "px")
else:
  print("- midline not available because: -")
  if result_morphology["termini"] != 2:
    print("termini", result_morphology["termini"])
  if result_morphology["branches"] != 0:
    print("branches", result_morphology["branches"])

In [ ]:
#@title Example application

#@markdown Flow cytometry-like cell cycle analysis of a cell line.

def cell_cycle_analysis(cell_line):
  results = {"kn": [], "kdna": [], "karea": [], "ndna": [], "narea": [], "length": [], "area": []}
  print(cell_line)
  fieldcell_list = tryptag.cell_list(cell_line)
  for cell in fieldcell_list:
    cell_image = tryptag.open_cell(cell_line, cell["field_index"], cell["cell_index"], width=-32)
    morphometry_analysis = tryptools.cell_morphology_analysis(cell_image)
    signal_analysis = tryptools.cell_signal_analysis(cell_image)
    if "distance" in morphometry_analysis:
      # area result
      results["area"].append(signal_analysis["cell_area"])
      # kn result
      kn = morphometry_analysis["count_kn"]
      if morphometry_analysis["count_kn"] not in ["1K1N", "2K1N", "2K2N"]:
        kn = "other"
      results["kn"].append(kn)
      # kdna and karea result
      kdna = 0
      karea = 0
      if "objects_k" in morphometry_analysis:
        for k in morphometry_analysis["objects_k"]:
          kdna += k["dna_sum"]
          karea += k["area"]
      results["kdna"].append(kdna)
      results["karea"].append(karea)
      # ndna and narea result
      ndna = 0
      narea = 0
      if "objects_n" in morphometry_analysis:
        for n in morphometry_analysis["objects_n"]:
          ndna += n["dna_sum"]
          narea += n["area"]
      results["ndna"].append(ndna)
      results["narea"].append(narea)
      # length result
      results["length"].append(morphometry_analysis["distance"][-1])
  return results

print(gene_id, terminus)
tryptag.gene_list["procyclic"]["Tb927.9.8570"]["n"]["plate"]
cell_cycle = cell_cycle_analysis(CellLine(gene_id, terminus, "procyclic"))

In [ ]:
#@title Example plotting

#@markdown Plot cell length against nuclear DNA content, grouped by kinetoplast and nucleus number.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.DataFrame(cell_cycle)

groups = df.groupby('kn')
for name, group in groups:
  plt.scatter(x=group['length'],y=group['ndna'],label=name)
plt.legend()
plt.xlabel("length")
plt.ylabel("ndna")
plt.show()